## Logistic Regression
#### 背景：
逻辑回归是经典的二分类机器学习模型，凭借可解释性强等优点，现在仍是主流二分类问题解决方案。

逻辑回归模型，首先将LinearRegression结果导入Sigmoid函数，实现对样本为positive概率的预测；
然后将预测概率与threshold进行比较，大于threshold预测为positive，反之为negative。

#### 目的：
1. 借助线性回归模型，探究主要数据预处理方法作用，及其对模型结果的影响
2. 探究矩阵运算对模型计算效率的提升作用
3. 探究模型过拟合的抑制方法和实用策略


In [ ]:
# 环境初始化
import numpy as np
import pandas as pd
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"  # 执行全部行输出命令

### 数据准备
浏览数据集描述信息后发现：
1. 数据集一共有13个特征、506个样本，且数据集中没有缺失数据；
2. 13个特征中，数值型特征11个，类别型特征2个（CHAS,RAD）

In [ ]:
# 加载数据
from sklearn.datasets import load_boston
data = load_boston()
print(data['DESCR'])
# data

In [ ]:
# 数据集特征探索
df = pd.DataFrame(data['data'],columns=data['feature_names'])
df['target']=data['target']
df.info(); df.head()

### 数据预处理
1. 数值型特征与类别型特征分离
2. 缺失值填充
3. 重复值处理
4. 异常值检测
5. 类别型特征编码
6. 数值型特征转换
7. 特征衍生与降维（可选）
8. 特征筛选（慎重）
9. 数据集分割

In [ ]:
# 数值型特征与类别型特征分离
df_cat = df[['CHAS','RAD']].astype('int').astype('category')
df_num = df.drop(columns=['CHAS','RAD','target'])
# print(df_cat.shape); print(df_num.shape)

#### 类别型特征编码主要有 ordinal encoding 和 one-hot encoding 两种方法

1. ordinal encoding编码
适用于处理类别间具有大小关系的顺序型类别特征，它按类别大小关系，给其赋予一个从1到n的正整数数值ID，将类别型特征转化成数值型哑变量。

2. one-hot encoding
适用于处理类别间不具有大小关系的分类型类别特征，它按特征值类别数量产生一个n维0-1稀疏向量，每种特征值由向量中对应维度为1、其它维度为0表示。
**使得任意两不同类别的编码向量之差相等，使得模型学习时，可以对每种类别一直同仁。**

3. 网传可以通过二进制编码的方法，转化分类型类别特征，得到与one-hot encoding同样的效果，并且向量维数少于one-hot、节省存储空间。
**这种说法是完全不对的，因为二进制编码与ordinal encoding的本质相同：编码向量间存在大小关系，任意不同类别编码向量间差值不等，模型学习时不能对特征的每种类别一视同仁**

In [ ]:
# 类别型特征编码
from sklearn.preprocessing import OneHotEncoder
onehot = OneHotEncoder()
nar_cat = onehot.fit_transform(df_cat).toarray()


In [ ]:
# 数值型特征描述性统计分析
df_num.describe()


#### 数值型特征转换主要有Feature Scaling和Non-linear transformation两大类方法

**Feature Scaling**

标准化

定义：对不同维度数值特征做线性变换，使得不同度量之间的特征具有可比性，它不改变原始数据的分布。

常用方法：
1. Normalization (min-max normalization)
2. Standardization(Z-score normalization)

Normalization是一个源于统计学的概念，它包含数据预处理中常说的Standardization和Normalization两个概念，它们都是Feature Scaling的一种方法。

*特别注意的是，稀疏数据一般不要执行标准化转换，因为Normalization会将一个多数元素为0的稀疏向量转化成一个多数元素不为0的密集特征向量，这会给模型运算带来巨大的负担。*

**Non-linear transformation**

归一化

定义：对不同维度数值特征做非线性变换，使各个特征维度对目标函数的影响权重一致，即将那些扁平分布的特征数据伸缩变换成类圆形分布，会改变原始数据的分布。

常用方法：
1. L2范数归一化
2. 对数变换
3. 指数变换

In [ ]:
# 数值型特征缩放
from sklearn.preprocessing import StandardScaler
standardized = StandardScaler()
nar_num = standardized.fit_transform(df_num)

#### 特征筛选

In [ ]:
# 特征线性相关性筛选
col_cat = ['CHAS'+str(i) for i in df_cat.CHAS.drop_duplicates()]+['RAD'+str(i) for i in df_cat.RAD.drop_duplicates()]
col_num = list(df_num.columns)
col = col_num + col_cat + ['target']
df_rul = pd.DataFrame(np.concatenate((nar_num, nar_cat, df[['target']].values), axis=1), columns=col)
df_rul.corr()['target'].sort_values(ascending=False)

# 获取标签数据集
y = df_rul[['target']].values.reshape(len(data['target']),1)

# 获取特征数据集1：保留全部特征
X_all = df_rul.drop(columns='target').values

# 获取特征数据集2：去除低相关性特征，保留中、高相关性特征
X_lcM = np.concatenate((nar_num, df_rul[['RAD24']].values), axis=1)

# 获取特征数据集3：仅保留高相关性特征
X_lcH = df_rul[['RM','PTRATIO','LSTAT']].values


#### 数据集分割

In [ ]:
# 定义分割函数
def data_split(data, test_ratio=0.2, val_ratio=0, index=0):
    if isinstance(index,int): index = np.random.choice(range(len(data)), size=len(data), replace=False)
    train_index = index[:int(len(data)*(1-val_ratio-test_ratio))]
    val_index = index[int(len(data)*(1-val_ratio-test_ratio)):int(len(data)*(1-test_ratio))]
    test_index = index[int(len(data)*(1-test_ratio)):]
    return data[train_index], data[test_index], data[val_index], index
#
y_train,y_test,_,_ = data_split(y,test_ratio=0.2)
X_all_train,X_all_test,_,X_index = data_split(X_all,test_ratio=0.2)
X_lcM_train,X_lcM_test,_,_ = data_split(X_lcM,test_ratio=0.2,index=X_index)
X_lcH_train,X_lcH_test,_,_ = data_split(X_lcH,test_ratio=0.2,index=X_index)


In [ ]:
# 模型定义
def LinReg_train(X,y,num_epochs,lr,lam,patient):
    num_sample,num_feature = X.shape
    # 初始化
    W_rec = [np.random.normal(0,1,(1,num_feature))]
    b_rec = [0]
    loss = [0]              # 每迭代损失函数值记录器
    loss_diff = [np.inf]    # 每迭代损失函数下降步长记录器
    j = 0                   # patient消耗程度记录器
    # 训练
    for i in range(num_epochs):
        W = W_rec[i]; b = b_rec[i]
        y_hat = X.dot(W.T)+b
        # MSE & L2
        # ls = float(np.sum((y_hat-y)**2)/(2*num_sample) + (lam/2)*W.dot(W.T))
        ls = np.sum((y_hat-y)**2)/(2*num_sample)
        # ls = (y_hat-y).dot((y_hat-y).T)/(2*num_sample)
        loss.append(ls)
        # 提前终止
        loss_diff.append(float(abs(loss[-1]-loss[-2])))
        if loss_diff[-1]>=min(loss_diff[:-1]): j+=1
        if j==patient:
            minLoss_index = loss.index(min(loss[1:]))
            return loss[1:minLoss_index+1], W_rec[minLoss_index-1], b_rec[minLoss_index-1]
        # 优化（GradientDescent & L2）
        W = W-lr*(y_hat-y).T.dot(X)/num_sample - lr*lam*W
        # W = W-lr*(y_hat-y).T.dot(X)/num_sample
        b = b-lr*np.mean(y_hat-y)
        W_rec.append(W); b_rec.append(b)
    return loss[1:], W_rec[-1], b_rec[-1]

# 预测
def LinReg_price(X,y,W,b,lam):
    num_sample,_ = X.shape
    y_hat = X.dot(W.T)+b
    # MSE
    ls = np.sum((y_hat-y)**2)/(2*num_sample)
    # ls = (y_hat-y).dot((y_hat-y).T)/(2*num_sample)
    return y_hat, ls


### 抑制过拟合
正则化和特征筛选是抑制过拟合的常用方法；

其中，正则化常用实现方法有参数L2/L1范数惩罚和提前终止等，它们可以组合使用也可以单独使用。

特征选择方法十分丰富，其目的是选出对预测标签最重要的一些特征，常用方法有线性相关性筛选、决策树/随机森林节点排序筛选和借助参数L1范数惩罚实现

基于本项目的试验结果，综合来看过拟合抑制方法的推荐排序是首先使用提前终止和参数L2范数惩罚，如果效果能不满足要求再谨慎使用特征筛选。

这是因为，提前终止没有需要校调的超参数，且在抑制过拟合的同时不会带来任何负面影响，但花无两样红，不得不说的是提前终止的过拟合抑制能力也远弱于另两种方法；

另一方面，谨慎使用特征筛选的原因是，特征筛选会损失信息，同时本项目的试验结果显示，同时使用提前终止、参数惩罚时，特征筛选会增加一些模型预测结果的不稳定性（重复执行程序，针对不同随机抽样数据，模型评估指标波动程度增加）；

通过合理设置参数L2范数惩罚项系数，可以在充分抑制过拟合的同时不损失特征信息、避免结果不稳定，所以结论：优先进行范数惩罚，最后再考虑特征筛选。


In [ ]:
# 超参数定义
num_epochs =1000000 #训练迭代次数
lr = 0.03           #学习率
lam = 10            #正则化惩罚系数
patient = 10        #提前停止容忍度
Num = 100          #重复试验次数

#
def Fun1(X_train,y_train,X_test,y_test,num_epochs,lr,lam,patient,Num):
    # 初始化
    ls=0; loss_=0
    #
    for _ in range(Num):
        # 训练
        loss_CV,W,b = LinReg_train(X_train,y_train,num_epochs,lr,lam,patient)
        # 模型效果评估
        ls+=loss_CV[-1]
        # 预测
        y_hat,loss = LinReg_price(X_test,y_test,W,b,3)
        loss_+=loss
    print(ls/Num, loss_/Num)

# 全特征
Fun1(X_all_train,y_train,X_all_test,y_test,num_epochs,lr,lam,patient,Num)
# 中、高相关特征
Fun1(X_lcM_train,y_train,X_lcM_test,y_test,num_epochs,lr,lam,patient,Num)
# 仅含高相关特征
Fun1(X_lcH_train,y_train,X_lcH_test,y_test,num_epochs,lr,lam,patient,Num)
